In [1]:
import numpy as np

def non_rated_values(Power_calc_MW, k_gen, Hg_max, Hg_min, Hg_average, turbine_rating_point):
    rte = 0.75
    head_loss_gen_ini = 0.05
    hours_gen = 8
    g = 9.81
    eff_t = 0.9
    eff_p = 0.91
    SortType = -1

    Q_gen_max_i = Power_calc_MW * 1000 / (Hg_max * (1 - head_loss_gen_ini) * g * eff_t)
    Q_gen_min_i = Power_calc_MW * 1000 / (Hg_min * (1 - head_loss_gen_ini) * g * eff_t)
    Q_gen_avg_i = Power_calc_MW * 1000 / (Hg_average * (1 - head_loss_gen_ini) * g * eff_t)

    if turbine_rating_point == "Average head":
        Q_gen_min_i = Q_gen_avg_i

    Q_pump_max_i = Power_calc_MW * 1000 / (Hg_max * (1 + head_loss_gen_ini * rte) * g / eff_p)
    Q_pump_min_i = Power_calc_MW * 1000 / (Hg_min * (1 + head_loss_gen_ini * rte) * g / eff_p)
    Q_pump_avg_i = Power_calc_MW * 1000 / (Hg_average * (1 + head_loss_gen_ini * rte) * g / eff_p)

    Hl_gen_max = k_gen * Q_gen_max_i ** 2
    Hl_gen_min = k_gen * Q_gen_min_i ** 2
    Hl_gen_avg = k_gen * Q_gen_avg_i ** 2

    Hl_pump_max = k_gen * Q_pump_max_i ** 2
    Hl_pump_min = k_gen * Q_pump_min_i ** 2
    Hl_pump_avg = k_gen * Q_pump_avg_i ** 2

    Htu_max, Htu_min, Htu_avg, Hpu_max, Hpu_min, Hpu_avg = calculate_head_values(
        Hg_max, Hg_min, Hg_average, Hl_gen_max, Hl_gen_min, Hl_gen_avg,
        Hl_pump_max, Hl_pump_min, Hl_pump_avg)

    Head_table = {
        "Hg_max": Hg_max, "Hg_min": Hg_min, "Hg_avg": Hg_average,
        "Q_gen_max": Q_gen_max_i, "Q_gen_min": Q_gen_min_i, "Q_gen_avg": Q_gen_avg_i,
        "Q_pump_max": Q_pump_max_i, "Q_pump_min": Q_pump_min_i, "Q_pump_avg": Q_pump_avg_i,
        "Hl_gen_max": Hl_gen_max, "Hl_gen_min": Hl_gen_min, "Hl_gen_avg": Hl_gen_avg,
        "Hl_pump_max": Hl_pump_max, "Hl_pump_min": Hl_pump_min, "Hl_pump_avg": Hl_pump_avg,
        "Htu_max": Htu_max, "Htu_min": Htu_min, "Htu_avg": Htu_avg,
        "Hpu_max": Hpu_max, "Hpu_min": Hpu_min, "Hpu_avg": Hpu_avg
    }

    return Head_table

def calculate_head_values(Hg_max, Hg_min, Hg_average, Hl_gen_max, Hl_gen_min, Hl_gen_avg,
                          Hl_pump_max, Hl_pump_min, Hl_pump_avg):
    Htu_max = Hg_max - Hl_gen_max
    Htu_min = Hg_min - Hl_gen_min
    Htu_avg = Hg_average - Hl_gen_avg

    Hpu_max = Hg_max + Hl_pump_max
    Hpu_min = Hg_min + Hl_pump_min
    Hpu_avg = Hg_average + Hl_pump_avg

    return Htu_max, Htu_min, Htu_avg, Hpu_max, Hpu_min, Hpu_avg

In [2]:
def WL_updated(u_fsl, u_mol, u_excav, u_fsl_vol, u_mol_vol, u_live_vol, u_live_vol_excav,
               l_fsl, l_mol, l_excav, l_fsl_vol, l_mol_vol, l_live_vol, l_live_vol_excav):
    WL_updated = np.zeros((7, 2))

    WL_updated[0, 0] = u_fsl
    WL_updated[1, 0] = u_mol
    WL_updated[2, 0] = u_excav
    WL_updated[3, 0] = u_fsl_vol
    WL_updated[4, 0] = u_mol_vol
    WL_updated[5, 0] = u_live_vol
    WL_updated[6, 0] = u_live_vol_excav

    WL_updated[0, 1] = l_fsl
    WL_updated[1, 1] = l_mol
    WL_updated[2, 1] = l_excav
    WL_updated[3, 1] = l_fsl_vol
    WL_updated[4, 1] = l_mol_vol
    WL_updated[5, 1] = l_live_vol
    WL_updated[6, 1] = l_live_vol_excav

    return WL_updated

In [3]:
def volumes_head_range_calc(WL_updated):
    u_fsl = WL_updated[0, 0]
    u_mol = WL_updated[1, 0]
    u_excav = WL_updated[2, 0]
    l_fsl = WL_updated[0, 1]
    l_mol = WL_updated[1, 1]
    l_excav = WL_updated[2, 1]
    
    u_fsl_vol = WL(u_fsl, 0)
    u_mol_vol = WL(u_mol, 0)
    l_fsl_vol = WL(l_fsl, 1)
    l_mol_vol = WL(l_mol, 1)
    
    u_live_vol = u_fsl_vol - u_mol_vol
    u_live_vol_excav = u_live_vol + u_excav
    
    l_live_vol = l_fsl_vol - l_mol_vol
    l_live_vol_excav = l_live_vol + l_excav
    
    WL_updated[3, 0] = u_fsl_vol
    WL_updated[4, 0] = u_mol_vol
    WL_updated[5, 0] = u_live_vol
    WL_updated[6, 0] = u_live_vol_excav
    
    WL_updated[3, 1] = l_fsl_vol
    WL_updated[4, 1] = l_mol_vol
    WL_updated[5, 1] = l_live_vol
    WL_updated[6, 1] = l_live_vol_excav
    
    return WL_updated

In [4]:
def volumes_r1(WL_updated, inputs, ur_y, ur_vol, lr_y, lr_vol):
    u_fsl_fix = inputs[5, 0]
    u_mol_fix = inputs[6, 0]
    l_fsl_fix = inputs[7, 0]
    l_mol_fix = inputs[8, 0]
    u_excav = WL_updated[2, 0]
    l_excav = WL_updated[2, 1]
    
    use_excav = inputs[1, 0]
    use_head_range = inputs[2, 0]
    use_power_solve = inputs[3, 0]
    
    ur_count = u_fsl_fix + u_mol_fix
    lr_count = l_fsl_fix + l_mol_fix
    
    case_id_solve = str(use_power_solve) + "_" + str(use_head_range)
    case_id_res = str(ur_count) + "_" + str(lr_count)
    
    SortType = -1
    
    if case_id_solve == "0_0":
        if case_id_res in ["2_2", "2_1", "1_2"]:
            WL_updated = volumes_simple_calc(case_id_res, WL_updated, inputs, ur_y, ur_vol, lr_y, lr_vol)
    elif case_id_solve == "1_0":
        WL_updated = volumes_power_calc(case_id_res, WL_updated, inputs, ur_y, ur_vol, lr_y, lr_vol)
    elif case_id_solve == "0_1":
        WL_updated = volumes_head_range_calc(WL_updated)
    
    return WL_updated

In [5]:
def non_rated_values(Power_calc_MW, k_gen, Hg_max, Hg_min, Hg_average, turbine_rating_point):
    rte = 0.75
    head_loss_gen_ini = 0.05
    hours_gen = 8
    g = 9.81
    eff_t = 0.9
    eff_p = 0.91
    SortType = -1
    
    Q_gen_max_i = Power_calc_MW * 1000 / (Hg_max * (1 - head_loss_gen_ini) * g * eff_t)
    Q_gen_min_i = Power_calc_MW * 1000 / (Hg_min * (1 - head_loss_gen_ini) * g * eff_t)
    Q_gen_avg_i = Power_calc_MW * 1000 / (Hg_average * (1 - head_loss_gen_ini) * g * eff_t)
    
    if turbine_rating_point == "Average head":
        Q_gen_min_i = Q_gen_avg_i
    
    Q_pump_max_i = Power_calc_MW * 1000 / (Hg_max * (1 + head_loss_gen_ini * rte) * g / eff_p)
    Q_pump_min_i = Power_calc_MW * 1000 / (Hg_min * (1 + head_loss_gen_ini * rte) * g / eff_p)
    Q_pump_avg_i = Power_calc_MW * 1000 / (Hg_average * (1 + head_loss_gen_ini * rte) * g / eff_p)
    
    Hl_gen_max = k_gen * Q_gen_max_i ** 2
    Hl_gen_min = k_gen * Q_gen_min_i ** 2
    Hl_gen_avg = k_gen * Q_gen_avg_i ** 2
    
    Hl_pump_max = k_gen * Q_pump_max_i ** 2
    Hl_pump_min = k_gen * Q_pump_min_i ** 2
    Hl_pump_avg = k_gen * Q_pump_avg_i ** 2
    
    delta_Q_max = 10
    delta_Q_tolerance = 0.1
    
    while delta_Q_max >= delta_Q_tolerance:
        Q_gen_max = Power_calc_MW * 1000 / ((Hg_max - Hl_gen_max) * g * eff_t)
        Q_gen_min = Power_calc_MW * 1000 / ((Hg_min - Hl_gen_min) * g * eff_t)
        Q_gen_avg = Power_calc_MW * 1000 / ((Hg_average - Hl_gen_avg) * g * eff_t)
        
        Q_pump_max = Power_calc_MW * 1000 / ((Hg_max + Hl_pump_max) * g / eff_p)
        Q_pump_min = Power_calc_MW * 1000 / ((Hg_min + Hl_pump_min) * g / eff_p)
        Q_pump_avg = Power_calc_MW * 1000 / ((Hg_average + Hl_pump_avg) * g / eff_p)
        
        delta_Q = [
            Q_gen_max - Q_gen_max_i,
            Q_gen_min - Q_gen_min_i,
            Q_gen_avg - Q_gen_avg_i,
            Q_pump_max - Q_pump_max_i,
            Q_pump_min - Q_pump_min_i,
            Q_pump_avg - Q_pump_avg_i
        ]
        
        delta_Q_max = max(map(abs, delta_Q))
        
        Q_gen_max_i, Q_gen_min_i, Q_gen_avg_i = Q_gen_max, Q_gen_min, Q_gen_avg
        Q_pump_max_i, Q_pump_min_i, Q_pump_avg_i = Q_pump_max, Q_pump_min, Q_pump_avg
        
    Htu_max, Htu_min, Htu_avg = Hg_max - Hl_gen_max, Hg_min - Hl_gen_min, Hg_average - Hl_gen_avg
    Hpu_max, Hpu_min, Hpu_avg = Hg_max + Hl_pump_max, Hg_min + Hl_pump_min, Hg_average + Hl_pump_avg
    
    Head_table = {
        "Hg_max": Hg_max, "Hg_min": Hg_min, "Hg_avg": Hg_average,
        "Q_gen_max": Q_gen_max, "Q_gen_min": Q_gen_min, "Q_gen_avg": Q_gen_avg,
        "Q_pump_max": Q_pump_max, "Q_pump_min": Q_pump_min, "Q_pump_avg": Q_pump_avg,
        "Hl_gen_max": Hl_gen_max, "Hl_gen_min": Hl_gen_min, "Hl_gen_avg": Hl_gen_avg,
        "Hl_pump_max": Hl_pump_max, "Hl_pump_min": Hl_pump_min, "Hl_pump_avg": Hl_pump_avg,
        "Htu_max": Htu_max, "Htu_min": Htu_min, "Htu_avg": Htu_avg,
        "Hpu_max": Hpu_max, "Hpu_min": Hpu_min, "Hpu_avg": Hpu_avg
    }
    
    return Head_table

In [6]:
def volumes_power_calc(case_id_res, WL, inputs, ur_y, ur_vol, lr_y, lr_vol):
    u_fsl_fix, u_mol_fix, l_fsl_fix, l_mol_fix = inputs[5][0], inputs[6][0], inputs[7][0], inputs[8][0]
    u_excav, l_excav = WL[2][0], WL[2][1]
    
    u_fsl_vol, u_mol_vol = WL[3][0], WL[4][0]
    l_fsl_vol, l_mol_vol = WL[3][1], WL[4][1]
    
    u_live_vol = u_fsl_vol - u_mol_vol
    u_live_vol_excav = u_live_vol + u_excav
    
    l_live_vol = l_fsl_vol - l_mol_vol
    l_live_vol_excav = l_live_vol + l_excav
    
    WL_updated = [
        [inputs[0][0]], [inputs[1][0]], [u_excav], [u_fsl_vol], [u_mol_vol], [u_live_vol], [u_live_vol_excav],
        [inputs[0][1]], [inputs[1][1]], [l_excav], [l_fsl_vol], [l_mol_vol], [l_live_vol], [l_live_vol_excav]
    ]
    
    return WL_updated

In [ ]:
volumes_power_calc,non_rated_values,volumes_r1,volumes_head_range_calc,WL_updated,calculate_head_values,non_rated_values

In [8]:

# Input parameters
Power_calc_MW = 10  # Power in MW
k_gen = 0.1  # Coefficient for head loss calculation
Hg_max = 100  # Maximum gross head
Hg_min = 50  # Minimum gross head
Hg_average = 75  # Average gross head
turbine_rating_point = "Average head"  # Options: "Average head" or "Minimum head"

# Call the function
head_values = non_rated_values(Power_calc_MW, k_gen, Hg_max, Hg_min, Hg_average, turbine_rating_point)

# Access calculated values
Htu_max = head_values["Htu_max"]  # Maximum turbine head
Htu_min = head_values["Htu_min"]  # Minimum turbine head
Hpu_max = head_values["Hpu_max"]  # Maximum pump head

In [9]:
print(Htu_max,Htu_min)

85.78555996308988 24.729884378826455
